## Workshop Homework

### Installing Required Dependencies

In [1]:
!pip install dlt[duckdb]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 373.9 kB/s eta 0:00:00 kB/s eta 0:00:0101
  Using cached wheel-0.42.0-py3-none-any.whl.metadata (2.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 967.6 kB/s eta 0:00:00MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 1.8 MB/s eta 0:00:004.3 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 31.5 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 3.3 MB/s eta 0:00:0031m173.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 10.0 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 3.5 MB/s eta 0:00:0031m143.1 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.7/540.7 kB 12.0 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.3 MB/s eta 0:00:00

### Using a Generator

In [2]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n**0.5
        n += 1

In [3]:
sum = 0
generator = square_root_generator(5)

for sqrt in generator:
    sum += sqrt

print(f"The sum of outputs of the generator for limit = 5 is {sum}")

The sum of outputs of the generator for limit = 5 is 8.382332347441762


In [6]:
x = 13
i = 1

generator = square_root_generator(x)
for sqrt in generator:
    if i == 13:
        print(f"The 13th number yielded is {sqrt}")
    i += 1

The 13th number yielded is 3.605551275463989


### Duckdb with dlt

In [7]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [8]:
import dlt
import duckdb

In [9]:
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators_people_1')

In [10]:
info = generators_pipeline.run(people_1(),
										table_name="people",
										write_disposition="replace")

In [11]:
print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.49 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators_people_1
The duckdb destination used duckdb:////workspaces/DEZoomcamp2024/notebooks/dlt_ipykernel_launcher.duckdb location to store data
Load package 1708007978.2417421 is LOADED and contains no failed jobs


In [12]:
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people table below:")

people_df = conn.sql("SELECT * FROM people").df()
display(people_df)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 people table below:


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1708007978.2417421,8r42E5N59vzH3A
1,2,Person_2,27,City_A,1708007978.2417421,iu2yGKrFMote1Q
2,3,Person_3,28,City_A,1708007978.2417421,zkdwjtYaYuRcjg
3,4,Person_4,29,City_A,1708007978.2417421,JMI+syFI9razdg
4,5,Person_5,30,City_A,1708007978.2417421,Fmq4ZkkX0EZlIQ


In [13]:
conn.sql("SELECT SUM(age) FROM people")

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      140 │
└──────────┘